In [ ]:
"""
Command Line Interface for the Autonomous Delivery Agent
"""

import argparse
import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))

from environment import GridEnvironment
from agent import DeliveryAgent
from utils import manhattan_distance, euclidean_distance

def main():
    parser = argparse.ArgumentParser(description='Autonomous Delivery Agent')
    parser.add_argument('--map', type=str, required=True, help='Map file path')
    parser.add_argument('--algorithm', choices=['bfs', 'ucs', 'astar', 'hillclimb', 'annealing'],
                       required=True, help='Path planning algorithm')
    parser.add_argument('--start', type=int, nargs=2, help='Start position (x y)')
    parser.add_argument('--goal', type=int, nargs=2, help='Goal position (x y)')
    parser.add_argument('--heuristic', choices=['manhattan', 'euclidean'], default='manhattan',
                       help='Heuristic for A* algorithm')
    parser.add_argument('--visualize', action='store_true', help='Visualize the path')
    parser.add_argument('--demo', action='store_true', help='Run demonstration mode')

    args = parser.parse_args()

    # Load environment
    env = GridEnvironment(0, 0)  # Dimensions will be set by load_from_file
    if not env.load_from_file(args.map):
        print(f"Error: Could not load map file {args.map}")
        return 1

    # Create agent
    agent = DeliveryAgent(env)

    if args.demo:
        run_demonstration(agent, env)
        return 0

    # Set start and goal
    start = args.start if args.start else env.agent_start
    goal = args.goal if args.goal else env.delivery_points[0] if env.delivery_points else (env.width-1, env.height-1)

    # Plan path
    result = agent.plan_path(args.algorithm, start, goal, heuristic=args.heuristic)

    if result['success']:
        print(f"Path found with cost {result['cost']}")
        print(f"Nodes expanded: {result['nodes_expanded']}")
        print(f"Planning time: {result['planning_time']:.4f} seconds")
        print(f"Path length: {len(result['path'])} steps")

        if args.visualize:
            env.visualize(start, result['path'])

        # Execute path
        execution_result = agent.execute_path(result['path'])
        if execution_result['success']:
            print("Delivery successful!")
            print(f"Fuel used: {execution_result['fuel_used']}")
            print(f"Time elapsed: {execution_result['time_elapsed']}")
        else:
            print("Delivery failed during execution")
            print(f"Reason: {execution_result.get('reason', 'Unknown')}")
    else:
        print("No path found!")

    return 0

def run_demonstration(agent, env):
    """Run a demonstration of dynamic replanning"""
    print("=== Autonomous Delivery Agent Demonstration ===")
    print("Environment loaded successfully")
    env.visualize()

    # Test all algorithms
    algorithms = ['bfs', 'ucs', 'astar', 'hillclimb']
    goal = env.delivery_points[0] if env.delivery_points else (env.width-1, env.height-1)

    print(f"\nTesting algorithms on path from {env.agent_start} to {goal}")
    print("-" * 50)

    for algorithm in algorithms:
        result = agent.plan_path(algorithm, env.agent_start, goal)
        status = "SUCCESS" if result['success'] else "FAILED"
        print(f"{algorithm.upper():<10} | Cost: {result['cost']:>6} | "
              f"Nodes: {result['nodes_expanded']:>6} | Time: {result['planning_time']:>7.4f}s | {status}")

    # Demonstrate dynamic replanning
    print("\n=== Dynamic Replanning Demonstration ===")

    # Add a dynamic obstacle
    env.dynamic_obstacles.append((5, 5, 5))  # Appears at time step 5

    # Plan initial path
    result = agent.plan_path('astar', env.agent_start, goal)
    if result['success']:
        print("Initial path planned successfully")

        # Simulate execution until obstacle appears
        for i, pos in enumerate(result['path']):
            if i >= 5:  # Obstacle appears at step 5
                if not env.is_valid_position(*pos, i):
                    print(f"Obstacle detected at step {i}! Position {pos} is blocked.")

                    # Replan
                    replan_result = agent.replan_for_dynamic_obstacle(result['path'][i-1], goal, 'astar')
                    if replan_result['success']:
                        print("Replanning successful! New path found.")
                        break
                    else:
                        print("Replanning failed!")
                        break

if __name__ == '__main__':
    sys.exit(main())